In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '293_eda'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
feature:
    name_bow_pca_dim: 10
    name_bow_word_th1: 5
    name_bow_word_th2: 3
    name_bow_th1_upper: 130
    name_bow_th2_upper: 1000
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])
pd.set_option('display.max_rows', 500)

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test_unknown.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


In [9]:
train = df_train.copy()
test = df_test.copy()
train['is_test'] = False
test['is_test'] = True
df_whole = pd.concat([train, test])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

# Target とカテゴリの時系列

In [13]:
pd.set_option('display.max_rows', 100)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
df_train.sort_values(['Year_of_Release'], ascending=False)[['Name', 'Genre', 'Year_of_Release', 'Global_Sales']].head(100)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Genre,Year_of_Release,Global_Sales
8358,Phantasy Star Online 2 Episode 4: Deluxe Package,Role-Playing,2017.0,1
8357,Phantasy Star Online 2 Episode 4: Deluxe Package,Role-Playing,2017.0,4
8356,Brothers Conflict: Precious Baby,Action,2017.0,1
8185,Samurai Warriors: Sanada Maru,Action,2016.0,1
8179,Period Cube: Torikago no Amadeus,Action,2016.0,1
8180,Republique,Action,2016.0,1
8181,Dead Rising,Action,2016.0,1
8182,Azure Striker Gunvolt,Action,2016.0,1
8183,Azure Striker Gunvolt: Striker Pack,Action,2016.0,1
8184,Worms: WMD,Action,2016.0,1


In [ ]:
df_train_score = df_train[~df_train['is_User_Score_tbd']]
df_train_score['User_Score'] = df_train_score['User_Score'].astype(float)

In [ ]:
SALES_COLUMNS = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
SCORE_COLUMNS = ['User_Score', 'Critic_Score', 'User_Count', 'Critic_Count']

In [ ]:
# rating system がはじまったのはいつから?
columns = [
    'User_Score', 'User_Count', 'Critic_Score', 'Critic_Count',
    'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'
]
df_train_score.groupby('Year_of_Release')[columns].mean()

In [ ]:
# Platform ごとのスコアは?
columns = [
    'User_Score', 'User_Count', 'Critic_Score', 'Critic_Count',
    'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'
]
df_train_score.groupby('Platform')[columns].mean()

In [ ]:
sns.pairplot(df_train[SCORE_COLUMNS])

In [ ]:
# データを Score あるなしで分けてみる

In [ ]:
us_na_tr = df_train['User_Score'].isna()
uc_na_tr = df_train['User_Count'].isna()
cs_na_tr = df_train['Critic_Score'].isna()
cc_na_tr = df_train['Critic_Count'].isna()
us_na_te = df_test['User_Score'].isna()
uc_na_te = df_test['User_Count'].isna()
cs_na_te = df_test['Critic_Score'].isna()
cc_na_te = df_test['Critic_Count'].isna()

df_no_score_train = df_train.loc[us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr]
df_no_score_test = df_test.loc[us_na_te & uc_na_te & cs_na_te & cc_na_te]
df_score_train = df_train.loc[~(us_na_tr & uc_na_tr & cs_na_tr & cc_na_tr)]
df_score_test = df_test.loc[~(us_na_te & uc_na_te & cs_na_te & cc_na_te)]

df_no_score_train.shape, df_score_train.shape, df_no_score_test.shape, df_score_test.shape

In [ ]:
user_score_agg = df_train.groupby(['Platform'])['User_Score'].agg(['mean', 'std'])
user_score_agg = change_column_name(
    user_score_agg, ['mean', 'std'], ['mean_user_score_by_platform', 'std_user_score_by_platform'])
critic_score_agg = df_train.groupby(['Platform'])['Critic_Score'].agg(['mean', 'std'])
critic_score_agg = change_column_name(
    critic_score_agg, ['mean', 'std'], ['mean_critic_score_by_platform', 'std_critic_score_by_platform'])

user_count_agg = df_train.groupby(['Platform'])['User_Count'].agg('mean')
user_count_agg = change_column_name(user_count_agg, 'User_Count', 'mean_user_count_by_platform')
critic_count_agg = df_train.groupby(['Platform'])['Critic_Count'].agg('mean')
critic_count_agg = change_column_name(critic_count_agg, 'Critic_Count', 'mean_critic_count_by_platform')

In [ ]:
us_na_tr = df_train['User_Score'].isna()
uc_na_tr = df_train['User_Count'].isna()
cs_na_tr = df_train['Critic_Score'].isna()
cc_na_tr = df_train['Critic_Count'].isna()
us_na_te = df_test['User_Score'].isna()
uc_na_te = df_test['User_Count'].isna()
cs_na_te = df_test['Critic_Score'].isna()
cc_na_te = df_test['Critic_Count'].isna()

In [ ]:
us_notna_cr_na = ~df_train['User_Score'].isna() & df_train['Critic_Score'].isna()
us_na_cr_notna = ~df_train['Critic_Score'].isna() & df_train['User_Score'].isna()
np.sum(us_notna_cr_na), np.sum(us_na_cr_notna)

In [ ]:
df_train = pd.merge(df_train, user_score_agg, how='left', on='Platform')
df_train = pd.merge(df_train, critic_score_agg, how='left', on='Platform')
df_train = pd.merge(df_train, user_count_agg, how='left', on='Platform')
df_train = pd.merge(df_train, critic_count_agg, how='left', on='Platform')
df_train.head()

In [ ]:
# mean_user_score_by_platform
# std_user_score_by_platform
# mean_critic_score_by_platform
# std_critic_score_by_platform

In [ ]:
def user_score_reconstruct(x):
    normalized_diff = (x['Critic_Score'] - x['mean_critic_score_by_platform']) / x['std_critic_score_by_platform']
    reconstructed = x['mean_user_score_by_platform'] + normalized_diff * x['std_user_score_by_platform']
    return reconstructed

df_train.loc[us_na_cr_notna, 'User_Score'] = df_train.loc[us_na_cr_notna].apply(user_score_reconstruct, axis=1)
df_train.loc[us_na_cr_notna, 'User_Count'] = df_train.loc[us_na_cr_notna, 'mean_user_count_by_platform']

In [ ]:
def critic_score_reconstruct(x):
    normalized_diff = (x['User_Score'] - x['mean_user_score_by_platform']) / x['std_user_score_by_platform']
    reconstructed = x['mean_critic_score_by_platform'] + normalized_diff * x['std_critic_score_by_platform']
    return reconstructed

df_train.loc[us_notna_cr_na, 'Critic_Score'] = df_train.loc[us_notna_cr_na].apply(critic_score_reconstruct, axis=1)
df_train.loc[us_notna_cr_na, 'Critic_Count'] = df_train.loc[us_notna_cr_na, 'mean_critic_count_by_platform']

In [ ]:
np.sum(~df_train['User_Score'].isna() & df_train['Critic_Score'].isna())

In [ ]:
np.sum(~df_train['Critic_Score'].isna() & df_train['User_Score'].isna())

In [ ]:
df_train = pd.read_csv(datadir / 'country_prob_train.csv')
df_test = pd.read_csv(datadir / 'country_prob_test_unknown.csv')

train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}


from mykaggle.feature.score_reconstruct import ScoreReconstruct

sr_train = ScoreReconstruct(train=True)
sr_test = ScoreReconstruct(train=False)

df_train_score = sr_train(df_train[['id']].copy(), train_others)
df_test_score = sr_test(df_test[['id']].copy(), test_others)

In [ ]:
for c in ['User_Score', 'Critic_Score']:
    df_train[c] = df_train_score[c]
    df_test[c] = df_test_score[c]

In [ ]:
# 確認
np.sum(~df_train['User_Score'].isna() & df_train['Critic_Score'].isna()), np.sum(~df_train['Critic_Score'].isna() & df_train['User_Score'].isna())

In [ ]:
# df_train.to_csv(datadir / 'country_prob_train_score.csv', index=False)
# df_test.to_csv(datadir / 'country_prob_test_unknown_score.csv', index=False)

In [ ]:
# df_train.to_csv(datadir / 'country_prob_train_score_only.csv', index=False)
# df_test.to_csv(datadir / 'country_prob_test_unknown_score_only.csv', index=False)

In [19]:
plb = 0.8406

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
plb - 0.8645

-0.023900000000000032